# MLOps 

This notebook is intended to help you get started managing your machine learning models and deployments in the H2O AI Cloud using python. 

**_This notebook assumes you have run 1 End to End Demo and have a project with a deployed model that exists with a specific name_**

* **Product Documentation:** https://docs.h2o.ai/mlops/
* **Python Documentation:** https://docs.h2o.ai/mlops/py-client-installing/

## Prerequistes

Update the `h2o_ai_cloud.py` file with the connection parameters for your H2O AI Cloud environemnt:
1. Login to your H2O AI Cloud environment
1. Click your username or avatar in the H2O AI Cloud navigation bar
1. Navigate to `CLI & API Access`
1. Use the variables from the `Accessing H2O AI Cloud APIs` section to populate the parameters

In [1]:
import requests
import json

from h2o_ai_cloud import mlops_client
import h2o_mlops_client

import pandas as pd

## Securely connect to the platform
We first connect to the H2O AI Cloud using our platform token to create a token provider object. We can then use this object to log into Steam and other APIs.

## Connect to MLOps

In [2]:
mlops = mlops_client()

Visit https://cloud.h2o.ai/auth/get-platform-token to get your platform token
Enter your platform token: ········


## Projects

### Create a project

In [3]:
mlops.storage.project.create_project({
    'project': {
         'description': "Demo project for a tutorial",
         'display_name': "my_test_project",
    }
})

{'project': {'created_time': datetime.datetime(2022, 7, 6, 1, 1, 47, 821112, tzinfo=tzutc()),
             'description': 'Demo project for a tutorial',
             'display_name': 'my_test_project',
             'id': 'aafa75a7-1e4b-46d7-a0d9-b9595ed806c4',
             'last_modified_time': datetime.datetime(2022, 7, 6, 1, 1, 47, 821112, tzinfo=tzutc()),
             'owner_id': '6366bc29-5f35-4659-b1ee-165b875313f6'}}

### List all projects you have access to

In [4]:
my_projects = mlops.storage.project.list_projects(body={}).project

for p in my_projects:
    print(p.id, p.display_name)

aafa75a7-1e4b-46d7-a0d9-b9595ed806c4 my_test_project
bbd95e60-74c2-4a95-856e-6d632dece23f Customer Churn
d2169b15-50cf-44da-9761-0dbb2a5617cd Telco Churn Predictions


### Delete a project 

In [5]:
for p in my_projects:
    if p.display_name == "my_test_project":
        mlops.storage.project.delete_project({'project_id': p.id})

In [6]:
for p in  mlops.storage.project.list_projects(body={}).project:
    print(p.id, p.display_name)

bbd95e60-74c2-4a95-856e-6d632dece23f Customer Churn
d2169b15-50cf-44da-9761-0dbb2a5617cd Telco Churn Predictions


### Select a specific project to work with
We have previously created this project using Driverless AI and added models to it in our first tutorial.

In [7]:
for p in my_projects:
    if p.display_name == "Telco Churn Predictions":
        project_id = p.id

## Experiments

### List experiments
Get a list of all experiments that are in our project

In [8]:
my_project_experiments = mlops.storage.experiment.list_experiments({
    'project_id': project_id,
}).experiment

for e in my_project_experiments:
    print(e.id, e.display_name)

bc7539d4-fcc5-11ec-bbbf-8af6970414ab Default Baseline


### Select an experiment

In [9]:
experiment = mlops.storage.experiment.get_experiment({
    'id': my_project_experiments[0].id, 
}).experiment
experiment

{'created_time': datetime.datetime(2022, 7, 6, 0, 53, 21, 186596, tzinfo=tzutc()),
 'display_name': 'Default Baseline',
 'id': 'bc7539d4-fcc5-11ec-bbbf-8af6970414ab',
 'last_modified_time': datetime.datetime(2022, 7, 6, 0, 53, 21, 186596, tzinfo=tzutc()),
 'metadata': None,
 'owner_id': '6366bc29-5f35-4659-b1ee-165b875313f6',
 'parameters': {'fold_column': '',
                'target_column': 'Churn?',
                'test_dataset_id': '',
                'training_dataset_id': '',
                'validation_dataset_id': '',
                'weight_column': ''},
 'statistics': {'training_duration': '117s'},
 'status': 'EXPERIMENT_STATUS_UNSPECIFIED',
 'tag': []}

## Deployments

### Deployment environment 
`Dev` and `Prod` are deployment environment tags that you can use for your model deployements. 

In [10]:
my_project_deployment_environments = mlops.storage.deployment_environment.list_deployment_environments(body={
    'project_id': project_id, 
}).deployment_environment

for de in my_project_deployment_environments:
    print(de.id, de.display_name)
    if de.display_name == "DEV":
        deployment_env_id = de.id

5ba7c218-8065-4e11-a819-c3bcbb991e1f DEV
9ad0c266-5913-4616-9057-9daa3188252f PROD


In [11]:
deployment_env = mlops.storage.deployment_environment.get_deployment_environment({
    'deployment_environment_id': deployment_env_id
}).deployment_environment
deployment_env

{'created_time': datetime.datetime(2022, 7, 6, 0, 53, 19, 295718, tzinfo=tzutc()),
 'deployment_target_name': 'kubernetes',
 'display_name': 'DEV',
 'id': '5ba7c218-8065-4e11-a819-c3bcbb991e1f',
 'last_modified_time': datetime.datetime(2022, 7, 6, 0, 53, 19, 295718, tzinfo=tzutc()),
 'project_id': 'd2169b15-50cf-44da-9761-0dbb2a5617cd'}

### Deployment Types
Deployment types help MLOps understand how traffic should be routed when new data is sent for predictions.

In [12]:
import h2o_mlops_client
h2o_mlops_client.StorageDeploymentType().allowable_values

['DEPLOYMENT_TYPE_UNSPECIFIED',
 'SINGLE_MODEL',
 'SHADOW_TRAFFIC',
 'SPLIT_TRAFFIC']

In [13]:
deployed_model = mlops.storage.deployment_environment.deploy({
    'deployment_environment_id': deployment_env_id,
    'experiment_id': my_project_experiments[0].id,
    'type': 'SINGLE_MODEL'
})

### Deployment health
Wait until our deployment has gone from launching to healthy

In [14]:
while mlops.deployer.deployment_status.get_deployment_status({
    'deployment_id': deployed_model.deployment.id
}).deployment_status.state != "HEALTHY":
    pass

In [15]:
deployment_status = mlops.deployer.deployment_status.get_deployment_status({
    'deployment_id': deployed_model.deployment.id
}).deployment_status

print(deployment_status.state)
print(deployment_status.scorer.sample_request.url)
print(deployment_status.scorer.score.url)

HEALTHY
https://model.cloud.h2o.ai/f517404f-f1c9-48fb-96d7-24049b24cc7f/model/sample_request
https://model.cloud.h2o.ai/f517404f-f1c9-48fb-96d7-24049b24cc7f/model/score


## Make predictions
Using the `https` library, we make predictions on new data

In [16]:
sample_request_as_text = requests.get(deployment_status.scorer.sample_request.url).text
sample_request = json.loads(sample_request_as_text)
sample_request

{'fields': ['State',
  'Account Length',
  'Area Code',
  "Int'l Plan",
  'VMail Plan',
  'VMail Message',
  'Day Mins',
  'Day Calls',
  'Day Charge',
  'Eve Mins',
  'Eve Calls',
  'Eve Charge',
  'Night Mins',
  'Night Calls',
  'Night Charge',
  'Intl Mins',
  'Intl Calls',
  'Intl Charge',
  'CustServ Calls'],
 'rows': [['text',
   '0',
   '0',
   'text',
   'text',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0']]}

In [17]:
fields = sample_request["fields"]
fields

['State',
 'Account Length',
 'Area Code',
 "Int'l Plan",
 'VMail Plan',
 'VMail Message',
 'Day Mins',
 'Day Calls',
 'Day Charge',
 'Eve Mins',
 'Eve Calls',
 'Eve Charge',
 'Night Mins',
 'Night Calls',
 'Night Charge',
 'Intl Mins',
 'Intl Calls',
 'Intl Charge',
 'CustServ Calls']

In [18]:
sample_row = sample_request["rows"][0]
sample_row

['text',
 '0',
 '0',
 'text',
 'text',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [19]:
new_row = ['10' + v for v in sample_row]
new_row

['10text',
 '100',
 '100',
 '10text',
 '10text',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100']

In [20]:
new_predictions = requests.post(
    url=deployment_status.scorer.score.url,
    json={
        'fields': fields,
        'rows': [sample_row, new_row]
    }
)

In [21]:
new_predictions

<Response [200]>

In [22]:
predictions_dict = json.loads(new_predictions.text)
predictions_dict

{'fields': ['Churn?.False.', 'Churn?.True.'],
 'id': 'bc7539d4-fcc5-11ec-bbbf-8af6970414ab',
 'score': [['0.96486217', '0.035137847'], ['0.55606866', '0.44393137']]}

In [23]:
pd.DataFrame(predictions_dict["score"], columns=predictions_dict["fields"])

,Churn?.False.,Churn?.True.
0,0.96486217,0.035137847
1,0.55606866,0.44393137


## Delete a deployment

In [24]:
mlops.storage.deployment.undeploy({'deployment_id': deployed_model.deployment.id})

{}